In [3]:
import sys
from pathlib import Path

current_dir = Path(__file__).resolve().parent if "__file__" in globals() else Path.cwd()
project_root = current_dir if (current_dir / "utils").exists() else current_dir.parent
project_root = project_root.resolve()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from utils.base import *

In [ ]:
def print_log(msg):
    print("{} - {}".format(datetime.now(), msg))

options = Options()
options.add_experimental_option("debuggerAddress", "localhost:9222")
driver = webdriver.Chrome(options=options)
driver.maximize_window()

# pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe" 

In [ ]:
ticket_number = 4
concert_date = "2024/12/07"
seat_category_key_word = "2800"
start_selling_ts = int((time.time()+5)*1000)
driver.get("https://tixcraft.com/activity/detail/24_xalive")
# driver.get("https://tixcraft.com/activity/detail/24_realive")
# driver.get("https://tixcraft.com/activity/detail/24_jihoon")

In [ ]:
# Wait for the ticket to start selling

last_print_ts = time.time()*1000 - 5000
while True:
    current_ts = time.time()*1000

    if current_ts >= start_selling_ts:
        break
    else:
        if current_ts - last_print_ts >= 1000:
            print_log("The ticket market will open in {} sec".format(int((start_selling_ts-current_ts)/1000)))
            last_print_ts = current_ts

    time.sleep(0.001)
print_log("Ticket market is opening".format(datetime.now()))
start = time.time()

# ==================================================================================================================================
# Click the "立即購票" Button
try:
    purchase_button = driver.find_element(By.XPATH, "//li[@class='buy']/a")
    ActionChains(driver).move_to_element(purchase_button).click(purchase_button).perform()
    print_log("Purchase button clicked.")
except Exception as e:
    print_log(f"Error clicking purchase button: {e}")
# ==================================================================================================================================
# Choose the event by the date we want 
WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.ID, "gameList")))

event_selected = False
rows = driver.find_elements(By.CSS_SELECTOR, "#gameList tbody tr")
for row in rows:
    date_text = row.find_element(By.TAG_NAME, "td").text
    if concert_date in date_text:
        purchase_button = row.find_element(By.CLASS_NAME, "btn-primary")
        purchase_button.click()
        event_selected = True
        print_log("Clicked '立即訂購' for the performance on 2024/12/07")
        break
    else:
        print_log("Performance not found.")

if not event_selected:
    purchase_button = row.find_element(By.CLASS_NAME, "btn-primary")
    purchase_button.click()
# ==================================================================================================================================
# Choose the seat catrgory we want 
WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.CLASS_NAME, "zone")))

available_tickets = driver.find_elements(By.CSS_SELECTOR, ".select_form_b a")
categories = driver.find_elements(By.CLASS_NAME, "category")
target_category = None

ticket_selected = False
for ticket in available_tickets:
    ticket_text = ticket.text    
    
    if seat_category_key_word in ticket_text:
        if "已售完" not in ticket_text:
            if "熱賣中" in ticket_text:
                ticket.click()
                ticket_selected = True
                print_log(f"Clicked on ticket: {ticket_text}")
                break
            else:
                ticket_remain = int(ticket_text.split(" ")[3])
                if ticket_remain >= 1:
                    ticket.click()
                    ticket_selected = True
                    print_log(f"Clicked on ticket: {ticket_text}")
                    break

if not ticket_selected:
    print_log("No available tickets were found.")
# ==================================================================================================================================
select_element = driver.find_element(By.CLASS_NAME, "form-select")
select = Select(select_element)
select.select_by_value("2")

terms_checkbox = driver.find_element(By.ID, "TicketForm_agree")
terms_checkbox.click()
end = time.time()

captcha_input = driver.find_element(By.ID, "TicketForm_verifyCode")
captcha_input.click()
print_log("Total cost: {} sec".format(round(end - start, 2)))
# # submit_button = driver.find_element(By.CSS_SELECTOR, "button[type='submit']")
# # submit_button.click()

In [ ]:
session = requests.Session()

# Define the cookies as a dictionary
cookies = {"SID": "ma36vl59u3r7o4su59i3siq123"}
session.cookies.update(cookies)

In [ ]:
while True:
    try:
        captcha_image = driver.find_element(By.ID, "TicketForm_verifyCode-image")
        captcha_url = captcha_image.get_attribute("src")

        next_img = True
        for i in range(3):
            response = session.get(captcha_url)
            img = Image.open(BytesIO(response.content))
            # img = img.convert("L")
            # img = img.filter(ImageFilter.MedianFilter())
            # enhancer = ImageEnhance.Contrast(img)
            # img = enhancer.enhance(2)  # Increase contrast
            # img = img.point(lambda x: 0 if x < 128 else 255)
            
            captcha_text = pytesseract.image_to_string(img).replace("\n","")
            if len(captcha_text) == 4:
                next_img = False
                break

        if next_img == True:
            captcha_image.click()
            print("Captcha image refreshed.")
        else:
            print("Captcha text:", captcha_text)
            break
    except:
        traceback.print_exc()
    time.sleep(0.1)


In [ ]:
captcha_text.replace("\n","")

In [ ]:
response = session.get(captcha_url)
img = Image.open(BytesIO(response.content))
img = img.convert("L")
img = img.filter(ImageFilter.MedianFilter())
enhancer = ImageEnhance.Contrast(img)
img = enhancer.enhance(2)  # Increase contrast
img = img.point(lambda x: 0 if x < 128 else 255)
img

In [ ]:
import requests
from PIL import Image
import pytesseract
from io import BytesIO

session = requests.Session()
cookies = {
    "SID": "ma36vl59u3r7o4su59i3siq123",
}
session.cookies.update(cookies)

# Fetch the CAPTCHA image
url = "https://tixcraft.com/ticket/captcha?v=672462398817f7.38958464"
response = session.get(url)

# Open the image
image = Image.open(BytesIO(response.content))

# Use OCR to extract text
captcha_answer = pytesseract.image_to_string(image)

print(f"The CAPTCHA answer is: {captcha_answer}")

In [ ]:
captcha_input = driver.find_element(By.ID, "TicketForm_verifyCode")
captcha_input.clear()
captcha_input.send_keys(captcha_answer)

In [ ]:
captcha_image = driver.find_element(By.ID, "TicketForm_verifyCode-image")
captcha_image.get_attribute("src")

In [ ]:
18 69 4
797660

In [ ]:
'https://tixcraft.com/ticket/captcha?v=67238c18e69ea4.00575875'
'https://tixcraft.com/ticket/captcha?v=67238c7976ce60.12752274'

In [ ]:
def custom_decode(hex_value):
    # Convert hexadecimal to bytes
    hex_bytes = bytes.fromhex(hex_value)
    
    # Try different custom mappings and shifts
    # 1. Use ASCII characters by adding a fixed offset
    shifted_chars = ''.join(chr((b % 26) + ord('a')) for b in hex_bytes[:4])
    
    # 2. Try using only the first 4 bytes interpreted as ASCII characters
    ascii_direct = ''.join(chr(b % 128) for b in hex_bytes[:4])

    # 3. Combine bytes and use modulo operations to simulate "random" mapping
    combined_chars = ''.join(chr((hex_bytes[i] + hex_bytes[i + 1]) % 26 + ord('a')) for i in range(0, min(len(hex_bytes) - 1, 4)))

    return {
        "shifted_chars": shifted_chars,
        "ascii_direct": ascii_direct,
        "combined_chars": combined_chars
    }

# Process each URL to extract the hex part and apply the custom decoding
for url, expected_answer in data.items():
    # Extract hexadecimal part before the decimal
    hex_part = url.split('=')[1].split('.')[0]
    results = custom_decode(hex_part)
    
    print(f"URL: {url}")
    print(f"Expected Answer: {expected_answer}")
    print("Generated Custom Decodings:")
    for method, result in results.items():
        print(f"  {method}: {result}")
    print("\n")